# Model Benchmarking with Kipoi
Benchmark tf-binding models in Kipoi. Follow the set up steps in README.md to activate the `kipoi-shared__envs__kipoi-py3-keras2-tf1` conda environment and to upload Gibbs sampling and GFlowNets models.

## Load packages
Load kipoi model zoo and evaluation packages.

In [1]:
import kipoi
import numpy as np
from sklearn.metrics import roc_auc_score

## Load data files
Use a labeled BED-format interval file and a genome fasta file.


In [2]:
intervals_file = 'example_data/intervals.tsv'
fasta_file = 'example_data/fasta.fa'
dataloader_kwargs = {'intervals_file': intervals_file, 'fasta_file': fasta_file}

Look at a few lines in the fasta file:

In [3]:
!head $fasta_file

>chr22
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN


Look at a few lines in the intervals file:

In [4]:
!head $intervals_file

chr22	20208963	20209064	0
chr22	29673572	29673673	0
chr22	28193720	28193821	0
chr22	43864274	43864375	0
chr22	18261550	18261651	0
chr22	7869409	7869510	0
chr22	49798024	49798125	0
chr22	43088594	43088695	0
chr22	35147671	35147772	0
chr22	49486843	49486944	0


The four columns in this file contain chromosomes, interval start coordinate, interval end coordinate, and the label. This file contains 2000 examples, 1000 positives and 1000 negatives.

Load the labels from the last column:

In [5]:
labels = np.loadtxt(intervals_file, usecols=(3,))

## pwm_HOCOMOCO
Simple PWM-scanning model PWM database: HOCOMOCO URL: http://hocomoco.autosome.ru/ Paper: Kulakovskiy et al 2015, HOCOMOCO: expansion and enhancement of the collection of transcription factor binding sites models: doi:10.1093/nar/gkv1249

In [9]:
## Load HOCOMOCO model
pwm_model_name = 'pwm_HOCOMOCO/human/NANOG'
pwm_model = kipoi.get_model(pwm_model_name)

16.4kB [00:03, 5.39kB/s]                                                                                                                                     


/Users/zijinhuang/opt/anaconda3/envs/kipoi-shared__envs__kipoi-py3-keras2-tf1/lib/python3.7/site-packages/keras/models.py:282: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [10]:
## Get HOCOMOCO predictions
pwm_predictions = pwm_model.pipeline.predict(dataloader_kwargs, batch_size=1000)

2it [00:00,  2.32it/s]


In [11]:
## Evaluate HOCOMOCO predictions
roc_auc_score(labels, pwm_predictions)

0.572797

## DeepBind
Abstract: Knowing the sequence specificities of DNA- and RNA-binding proteins is essential for developing models of the regulatory processes in biological systems and for identifying causal disease variants. Here we show that sequence specificities can be ascertained from experimental data with 'deep learning' techniques, which offer a scalable, flexible and unified computational approach for pattern discovery. Using a diverse array of experimental data and evaluation metrics, we find that deep learning outperforms other state-of-the-art methods, even when training on in vitro data and testing on in vivo data. We call this approach DeepBind and have built a stand-alone software tool that is fully automatic and handles millions of sequences per experiment. Specificities determined by DeepBind are readily visualized as a weighted ensemble of position weight matrices or as a 'mutation map' that indicates how variations affect binding within a specific sequence.

In [6]:
## Load DeepBind model
deepbind_model_name = 'DeepBind/Homo_sapiens/TF/D00786.001_ChIP-seq_NANOG'
deepbind_model = kipoi.get_model(deepbind_model_name)

Already up to date.
Using downloaded and verified file: /Users/zijinhuang/.kipoi/models/DeepBind/downloaded/model_files/Homo_sapiens/TF/D00786.001_ChIP-seq_NANOG/arch/24ee28e3c0e5aa02575959ca440bb043
Using downloaded and verified file: /Users/zijinhuang/.kipoi/models/DeepBind/downloaded/model_files/Homo_sapiens/TF/D00786.001_ChIP-seq_NANOG/weights/8d27359f94cb8b2753c1912234638573


Using TensorFlow backend.


2022-05-31 07:46:16.366272: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2022-05-31 07:46:16.470349: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7fde53b6e6b0 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-05-31 07:46:16.470369: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [7]:
## Get DeepBind predictions
deepbind_predictions = deepbind_model.pipeline.predict(dataloader_kwargs, batch_size=1000)

2it [00:04,  2.01s/it]


In [8]:
## Evaluate DeepBind predictions
roc_auc_score(labels, deepbind_predictions)

0.6805905000000001

## lsgkm-SVM
lsgkm-SVM model trained on the ENCODE datasets - no sample number limitation as opposed to gkm-SVM publication. Datasets downloaded from http://ftp.ebi.ac.uk/pub/databases/ensembl/encode/integration_data_jan2011/byDataType/peaks/jan2011/spp/optimal/hub/. All files were then processed using genNullSeqs(...,nMaxTrials=20,xfold=1,genomeVersion="hg19") from the gkmSVM package version 0.79.0. For training all chromosomes except chr8 and chr9 were used.

In [12]:
## Load lsgkm-SVM model
lsgkm_svm_model_name = 'lsgkm-SVM/Tfbs/Nanogsc33759/H1hesc/Haib_V0416102'
lsgkm_svm_model = kipoi.get_model(lsgkm_svm_model_name)

2.59MB [00:10, 250kB/s]                                                                                                                                      


In [31]:
## Get lsgkm-SVM predictions
lsgkm_svm_predictions = lsgkm_svm_model.pipeline.predict(dataloader_kwargs, batch_size=1000)

0it [00:00, ?it/s]Wrong number of arguments [7].
1it [00:01,  1.31s/it]


Usage: gkmpredict [options] <test_seqfile> <model_file> <output_file>

 score test sequences using trained gkm-SVM

Arguments:
 test_seqfile: sequence file for test (fasta format)
 model_file: output of gkmtrain
 output_file: name of output file

Options:
 -v <0|1|2|3|4>  set the level of verbosity (default: 2)
                   0 -- error msgs only (ERROR)
                   1 -- warning msgs (WARN)
                   2 -- progress msgs at coarse-grained level (INFO)
                   3 -- progress msgs at fine-grained level (DEBUG)
                   4 -- progress msgs at finer-grained level (TRACE)
-T <1|4|16>      set the number of threads for parallel calculation, 1, 4, or 16
                 (default: 1)



Wrong number of arguments [7].
2it [00:01,  1.09it/s]


Usage: gkmpredict [options] <test_seqfile> <model_file> <output_file>

 score test sequences using trained gkm-SVM

Arguments:
 test_seqfile: sequence file for test (fasta format)
 model_file: output of gkmtrain
 output_file: name of output file

Options:
 -v <0|1|2|3|4>  set the level of verbosity (default: 2)
                   0 -- error msgs only (ERROR)
                   1 -- warning msgs (WARN)
                   2 -- progress msgs at coarse-grained level (INFO)
                   3 -- progress msgs at fine-grained level (DEBUG)
                   4 -- progress msgs at finer-grained level (TRACE)
-T <1|4|16>      set the number of threads for parallel calculation, 1, 4, or 16
                 (default: 1)



In [33]:
## Evaluate lsgkm-SVM predictions
roc_auc_score(labels, lsgkm_svm_predictions)

0.73245608


## DeepSEA
This CNN is based on the DeepSEA model from Zhou and Troyanskaya (2015). The model has been converted to a pytorch model on a modified version of https://github.com/clcarwin/convert_torch_to_pytorch Use this model only for predictions of sequences, not for variant effect prediction. The model automatically generates reverse-complement and averages over forward and reverse-complement to results from the website. To predict variant effects use the DeepSEA/variantEffects model. It categorically predicts 919 cell type-specific epigenetic features from DNA sequence. The model is trained on publicly available ENCODE and Roadmap Epigenomics data and on DNA sequences of size 1000bp. The input of the tensor has to be (N, 4, 1, 1000) for N samples, 1000bp window size and 4 nucleotides. Per sample, 919 probabilities of a specific epigentic feature will be predicted.

In [24]:
## Load DeepSEA model
deepsea_model_name = 'DeepSEA/predict'
deepsea_model = kipoi.get_model(deepsea_model_name)

Using downloaded and verified file: /Users/zijinhuang/.kipoi/models/DeepSEA/predict/downloaded/model_files/weights/89e640bf6bdbe1ff165f484d9796efc7


/Users/zijinhuang/opt/anaconda3/envs/kipoi-shared__envs__kipoi-py3-keras2-tf1/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [25]:
## Get DeepSEA predictions
deepsea_predictions = deepsea_model.pipeline.predict(dataloader_kwargs, batch_size=1000)

2it [02:15, 67.94s/it]


In [34]:
## Evaluate DeepSEA predictions
roc_auc_score(labels, deepsea_predictions)

0.8234910


## BPNet
Train and interpret base-resolution deep neural networks trained on functional genomics data such as ChIP-nexus or ChIP-seq. It addresses the problem of pinpointing the regulatory elements in the genome.

In [35]:
## Load BPNet model
bpnet_model_name = 'BPNet-OSKN'
bpnet_model = kipoi.get_model(bpnet_model_name)

Using downloaded and verified file: /Users/zijinhuang/.kipoi/models/BPNet-OSKN/downloaded/model_files/model_file/bbe883baef261877bfad07d05feb627d


In [36]:
## Get BPNet predictions
bpnet_predictions = bpnet_model.pipeline.predict(dataloader_kwargs, batch_size=1000)

In [43]:
## Evaluate BPNet predictions
roc_auc_score(labels, bpnet_predictions)

0.85910128


## GFlowNets
Save MCMC model and Gibbs sampling model according to steps in README.

In [41]:
## Load GFlowNets model
gflownets_model_name = 'GFlowNets'
gflownets_model = kipoi.get_model(gflownets_model_name)

In [42]:
## Get GFlowNets predictions
gflownets_predictions = gflownets_model.pipeline.predict(dataloader_kwargs, batch_size=1000)

In [44]:
## Evaluate GFlowNets predictions
roc_auc_score(labels, gflownets_predictions)

0.563295


## Gibbs Sampling
Save MCMC model and Gibbs sampling model according to steps in README.

In [45]:
## Load Gibbs sampling model
gibbs_sampling_model_name = 'Gibbs-Sampling'
gibbs_sampling_model = kipoi.get_model(gibbs_sampling_model_name)

In [46]:
## Get Gibbs Sampling predictions
gibbs_sampling_predictions = gibbs_sampling_model.pipeline.predict(dataloader_kwargs, batch_size=1000)

In [47]:
## Evaluate Gibbs Sampling predictions
roc_auc_score(labels, gibbs_sampling_predictions)

0.5289708
